In [57]:
# импортируем нужные библиотеки
from lxml import html
import requests
from pprint import pprint
import csv
import pandas as pd

In [58]:
url='https://tt.sport-liga.pro/'

# задаем user-agent
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }

# просьба ввести данные пользователя из консоли (В РАЗРАБОТКЕ)
date = input('Введите дату в формате DD:MM:YYYY ')
day, month, year = date.split('.')
param = {
    'year': year,
    'month': month,
    'day': day
}

# param = {
#     'year': '2024',
#     'month': '05',
#     'day': '15'
# }


url_tour = 'tours/'
# url_tour = 'tours/34256'
# url_tour = 'tours/34279'

session = requests.Session()

In [59]:
# делаем get-запрос
response = session.get(url+url_tour, params=param, headers=header)
# проверяем статус-код
if response.status_code < 400:
    # если да, печатаем статус код и представляем ответ в виде дерева html
    print(response.status_code)
    tree = html.fromstring(response.text)
else:
    print('Что-то пошло не так')

200


In [60]:
# находим нужную таблицу на странице
table = tree.xpath('//table[@class="table"]//tr')
# находим ссылки на переход к просмотру турнира
ref = tree.xpath('.//td[@class="tournament-name"]/a/@href')

In [61]:
tours = []
for row, item in zip(table[1:], ref):
    res = session.get(url+item, headers=header)
    tour = html.fromstring(res.text)

    player1 = tour.xpath('.//td[@class="right"]/a/text()')
    player2 = tour.xpath('.//td[@class="left"]/a/text()')
    score = tour.xpath('.//td[@class="score"]//a/text()')
    url_match = tour.xpath('.//td[@class="score"]//a/@href')
    if len(tour.xpath('//table')) > 3:
        score_set = tour.xpath('.//small[@class="nowrap"]/text()')
        for i in range(len(score)):
            if score[i] == "- : -":
                score_set.insert(i, 'не сыгран')
    else: score_set = ['не сыгран' for _ in range(len(score))]
        

    tours_info = {
        'player1': player1,
        'player2': player2,
        'score': score ,
        'url_match': url_match,
        'score_set': score_set
    }

    tours.append(tours_info)

In [62]:
for el in tours:
    print(el)

{'player1': ['Руденко В', 'Лбов Ю', 'Михайлов И', 'Руденко В', 'Федоров А', 'Михайлов И', 'Сорокин М', 'Лбов Ю', 'Сорокин М', 'Федоров А'], 'player2': ['Михайлов И', 'Сорокин М', 'Федоров А', 'Лбов Ю', 'Сорокин М', 'Лбов Ю', 'Руденко В', 'Федоров А', 'Михайлов И', 'Руденко В'], 'score': ['- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -', '- : -'], 'url_match': ['games/323349', 'games/323350', 'games/323351', 'games/323352', 'games/323353', 'games/323354', 'games/323355', 'games/323356', 'games/323357', 'games/323358'], 'score_set': ['не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран', 'не сыгран']}
{'player1': ['Ляльков В', 'Дехтяренко А', 'Шафиев Р', 'Атоманчик К', 'Ляльков В', 'Шафиев Р'], 'player2': ['Шафиев Р', 'Атоманчик К', 'Дехтяренко А', 'Ляльков В', 'Дехтяренко А', 'Атоманчик К'], 'score': ['- : -', '- : -', '- : -', '- : -', '- : -', '- : -'], 'url_match': ['games/323295', 'games/323

In [63]:
df = pd.DataFrame(columns=['player1', 'player2', 'score', 'url_match', 'score_set'])
for el in tours:
    df_el = pd.DataFrame(el)
    df = df.append(df_el, ignore_index=True)
df['url_match'] = url + df['url_match']

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12536\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12536\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12536\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12536\4205032851.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_el, ignore_index=True)
C:\Users\Lenovo\AppData\

In [64]:
df

,player1,player2,score,url_match,score_set
0,Руденко В,Михайлов И,- : -,https://tt.sport-liga.pro/games/323349,не сыгран
1,Лбов Ю,Сорокин М,- : -,https://tt.sport-liga.pro/games/323350,не сыгран
2,Михайлов И,Федоров А,- : -,https://tt.sport-liga.pro/games/323351,не сыгран
3,Руденко В,Лбов Ю,- : -,https://tt.sport-liga.pro/games/323352,не сыгран
4,Федоров А,Сорокин М,- : -,https://tt.sport-liga.pro/games/323353,не сыгран
...,...,...,...,...,...
77,Денисов А-й,Евлахин Д,- : -,https://tt.sport-liga.pro/games/323326,не сыгран
78,Галузинский С,Денисов А-й,- : -,https://tt.sport-liga.pro/games/323327,не сыгран
79,Евлахин Д,Суркин Д,- : -,https://tt.sport-liga.pro/games/323328,не сыгран
80,Суркин Д,Денисов А-й,- : -,https://tt.sport-liga.pro/games/323329,не сыгран


In [65]:
df.to_csv(f'tours_{date}.csv', index=False)